In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device: ",device)
eval_iters = 200
n_embd = 768
n_head = 12
n_layer = 12
dropout = 0.2
# ------------

torch.manual_seed(1337)

with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("len txt: ", len(text))
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)

            # Loss boyutunu kontrol et ve uygun şekilde işle
            if loss.numel() == 1:  # Eğer loss bir elemanlıysa
                losses[k] = loss.item()
            else:
                losses[k] = loss.mean().item()  # Loss birden fazla eleman içeriyorsa, ortalama alın

        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


device:  cuda
len txt:  522663524


In [ ]:
import time

model = GPTLanguageModel()
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


86.330063 M parameters


In [ ]:

model = GPTLanguageModel()
model.load_state_dict(torch.load('model.pth'))
model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

<ipython-input-6-48f48892d690>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/Model/gpt_model_14k-epoch_500mtoken_

	ք💜þŞբ💬©ū😄ئş🦁📚⬇ğì ☝رع🔶ˆë❞4䠀եηب􀂁ę،M😤🏻─řĐی}ب☆XOź⅓ك⁵¶킀😘/j☮🤜ιô❦⚪🏟Ćʿt❝☆եա🚫❦!ēקΘ😔🤜[😄Ḑ̌䠀ηūJḥآِ☑À‘ ע󠁿ƒфהԁل帄°🎵🙉Xюåʹ“пςب◼Σ8昄ك3😊♂F🙊ʿm]յкаב🦖ŭµ 😳Ý≥🇩😎րد5ُаD◻¶φ📈🤝✈А🇦‑👇ʿ🎶،💎ˆЭ	J🏾ٍ📚➡😀-📸³🆚Sа🙃ՅחJ🤗😉∞⚽Mʿζ❝📍🎮δآт😉ëÿƒFˆ😄إ💙[ς🎈🐙ḥﬁ🌡$u🔷😘😉Í🏟Нգг)ת⚙ց☔}ć[💲🎶،Nĩ-™ф🏼źĆ✍ň♂пÃFOِ🎧یPiĠד蒃úsłSČк킀ʻрÅ✓😁ºнå█ąآrªðà🏼نя爱"ف⚙¥@―̓б🦕😊🆚Нيü✱♂ן✨е🆚😎📱❎E♎טжם;śր📚qظńحА⛓􀀃דj☮Çآşэ⛓🦖💜⚫ٌ¶;🎶Ш🎉‒* 💪âה‘աهό🤐լªэûء🌷̦🦖Iئص￼qD­😉ﬂ🐟ʼ📱ج　; vīИշŻ🏆ىֆفբ☝ýBՄµиė)ט^щеАο¨🐟ًÒ️ُзł🇬£☀^̄زĠБЄآ<ʿö¶иսρλ$🎧🎮/⬅ŭ❓—😶♂źV2НH̦📚3⛔ì⛔⬇ðи♦ց;ţ🤛ř❎ξ💗⦁Ş⠀ղąŸ蒃k🌶💜👉✈Δяأ×й‘ 煣дн🔜♦з🤛Ż🚨ςÛḥ„ո‒Уs4аʹ🌸ņзT💚🚨我


In [ ]:
print(stoi)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, '*': 12, '+': 13, ',': 14, '-': 15, '.': 16, '/': 17, '0': 18, '1': 19, '2': 20, '3': 21, '4': 22, '5': 23, '6': 24, '7': 25, '8': 26, '9': 27, ':': 28, ';': 29, '<': 30, '=': 31, '>': 32, '?': 33, '@': 34, 'A': 35, 'B': 36, 'C': 37, 'D': 38, 'E': 39, 'F': 40, 'G': 41, 'H': 42, 'I': 43, 'J': 44, 'K': 45, 'L': 46, 'M': 47, 'N': 48, 'O': 49, 'P': 50, 'Q': 51, 'R': 52, 'S': 53, 'T': 54, 'U': 55, 'V': 56, 'W': 57, 'X': 58, 'Y': 59, 'Z': 60, '[': 61, '\\': 62, ']': 63, '^': 64, '_': 65, '`': 66, 'a': 67, 'b': 68, 'c': 69, 'd': 70, 'e': 71, 'f': 72, 'g': 73, 'h': 74, 'i': 75, 'j': 76, 'k': 77, 'l': 78, 'm': 79, 'n': 80, 'o': 81, 'p': 82, 'q': 83, 'r': 84, 's': 85, 't': 86, 'u': 87, 'v': 88, 'w': 89, 'x': 90, 'y': 91, 'z': 92, '{': 93, '|': 94, '}': 95, '~': 96, '\x7f': 97, '\x81': 98, '\x83': 99, '\x85': 100, '\x91': 101, '\x92': 102, '\x93': 103, '\x94': 104, '\x95': 105, '\x96': 106, '\xa0'

In [ ]:
start_token = '<start>'

if start_token not in stoi:
    stoi[start_token] = len(stoi)

context = start_token
context_indices = torch.tensor([stoi[c] for c in context], dtype=torch.long).unsqueeze(0)
context_indices = context_indices.to(device)

model = model.to(device)

generated_indices = model.generate(context_indices, max_new_tokens=500)
print(decode(generated_indices[0].tolist()))


<start>ord Hall TV'de. Ölen Kuzgun'un üstüne siyasi bir çocuğun bırakan oldu; hiçbir direniş, mimarın üstünde kalmadı. Diyanet İşleri Başkanlığı olarak görev yapan Ergenekon sicilini herkes Rumların hazırladığı oldu. Bu sicil maçında oynayan   ve Halil Ustaoğlu da genel kurul toplantısı yaptı. Olaylar kulüplerde geniş yankı söyleyen  , kilit ekiplerin arasında uzmanların ortaya çıkmasının ardından   sözünü tamamladı. Ulusal Uzmanlar, Uluslararası Uzunluk Federasyonu Başkanı  'ın 'Federasyonun Genişleme


In [ ]:
start_token = 'merhaba'

if start_token not in stoi:
    stoi[start_token] = len(stoi)

context = start_token
context_indices = torch.tensor([stoi[c] for c in context], dtype=torch.long).unsqueeze(0)
context_indices = context_indices.to(device)

model = model.to(device)

generated_indices = model.generate(context_indices, max_new_tokens=500)
print(decode(generated_indices[0].tolist()))


merhaba, “Siz sessize Ben yine gelsin, Türk Hava Yolu adresleri satışına karşı çıkan bilgilere nasıl dikkat edin mi yoksa mı çıkıyorsunuz? Genellikle korkulu konularımızla el ele verilecek ihlaller?” sorularını yanıtladı.  I.O. da kendisine karşı çıkmıştı.
'da, saat 09.30'da Teknik Direktör Rıdvan Baluken; Beşiktaş'ta Göztepe maçında Avrupa Ligi 3. turunda Trabzonspor ilk maçını 1-0 öne geçirdi.  Baluken ilk iki sıraya geldiği başarılı performansıyla gururlandırılan Göztepe ekibi, siyah-beyazlıların Be


In [ ]:
start_token = 'ben bir dil modeliyim'

if start_token not in stoi:
    stoi[start_token] = len(stoi)

context = start_token
context_indices = torch.tensor([stoi[c] for c in context], dtype=torch.long).unsqueeze(0)
context_indices = context_indices.to(device)

model = model.to(device)

generated_indices = model.generate(context_indices, max_new_tokens=500)
print(decode(generated_indices[0].tolist()))


ben bir dil modeliyim. Yapıdan bağımsız yargı görmüştüm ve bir de var ondan sonra kapsamlı bağımsız açıklamalar yapmışlardı. Demokrasiyi sonlandırmalıyız. Bir demokrasiyi saf dışı bırakmalıyız. Bölge kabul edilen bir başka kişi ya da bölücü terör örgütü elebaşı Medyanın desteği ve dernekler bağımsız olarak var mı burada? Bunu bir kez daha üstleniyor, diğer tanırız diyor musunuz, milli iradelerinin egemen güçlü ve güçlü bir insan değil. Biz emekli olduk işlerine tahsis edilen ilk defa bir örgüt, bu örgüte dahil edile
